# Extended RNA-Seq Analysis Training Demo

## Overview

For simplicity and time, The short tutorial workflow uses truncated and partial run data from the Cushman et al., project.

The tutorial repeats the short tutorial, but with the full fastq files and includes some extra steps, such as how to download and prepare the transcriptome files used by salmon, alternate ways to navigate the NCBI databases for annotation or reference files you might need, and how to combine salmon outputs at the end into a single genecount file.

Full fastq files can be rather large, and so the downloading, extracting, and analysis of them means this tutorial can take over 1 hour 45 minutes to run the code fully. This is part of the reason we have a short and easy introductory tutorial, and this longer more full tutorial for those interested.

If this is too lengthy feel free to move on to the snakemake tutorial or the DEG analysis tutorial -- all the files used in the DEG tutorial were created using this extended tutorial workflow.

![RNA-Seq workflow](images/rnaseq-workflow.png)

### STEP 1: Install Mambaforge and then install snakemake using bioconda.

First install Mambaforge.


In [1]:
!curl -L -O https://github.com/conda-forge/miniforge/releases/latest/download/Mambaforge-$(uname)-$(uname -m).sh
!bash Mambaforge-$(uname)-$(uname -m).sh -b -u -p $HOME/mambaforge
!export PATH="$HOME/mambaforge/bin:$PATH"
!date +"%T"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 88.1M  100 88.1M    0     0  24.5M      0  0:00:03  0:00:03 --:--:-- 28.2M
PREFIX=/home/jupyter/mambaforge
Unpacking payload ...
Extracting "brotlipy-0.7.0-py39hb9d737c_1004.tar.bz2"
Extracting "openssl-1.1.1o-h166bdaf_0.tar.bz2"
Extracting "reproc-cpp-14.2.3-h9c3ff4c_0.tar.bz2"
Extracting "libmambapy-0.22.1-py39hd55135b_1.tar.bz2"
Extracting "libev-4.33-h516909a_1.tar.bz2"
Extracting "lzo-2.10-h516909a_1000.tar.bz2"
Extracting "c-ares-1.18.1-h7f98852_0.tar.bz2"
Extracting "libffi-3.4.2-h7f98852_5.tar.bz2"
Extracting "tzdata-2022a-h191b570_0.tar.bz2"
Extracting "six-1.16.0-pyh6c4a22f_0.tar.bz2"
Extracting "libiconv-1.16-h516909a_0.tar.bz2"
Extracting "reproc-14.2.3-h7f988

Next, using mambaforge and bioconda, install the tools that will be used in this tutorial.

In [2]:
!$HOME/mambaforge/bin/mamba install -y -c conda-forge -c bioconda trimmomatic
!$HOME/mambaforge/bin/mamba install -y -c conda-forge -c bioconda fastqc
!$HOME/mambaforge/bin/mamba install -y -c conda-forge -c bioconda multiqc
!$HOME/mambaforge/bin/mamba install -y -c conda-forge -c bioconda salmon
!$HOME/mambaforge/bin/mamba install -y -c conda-forge -c bioconda sra-tools
!$HOME/mambaforge/bin/mamba install -y -c conda-forge -c bioconda parallel-fastq-dump
!$HOME/mambaforge/bin/mamba install -y -c conda-forge -c bioconda entrez-direct
!$HOME/mambaforge/bin/mamba install -y -c conda-forge -c bioconda gffread


                  __    __    __    __
                 /  \  /  \  /  \  /  \
                /    \/    \/    \/    \
███████████████/  /██/  /██/  /██/  /████████████████████████
              /  / \   / \   / \   / \  \____
             /  /   \_/   \_/   \_/   \    o \__,
            / _/                       \_____/  `
            |/
        ███╗   ███╗ █████╗ ███╗   ███╗██████╗  █████╗
        ████╗ ████║██╔══██╗████╗ ████║██╔══██╗██╔══██╗
        ██╔████╔██║███████║██╔████╔██║██████╔╝███████║
        ██║╚██╔╝██║██╔══██║██║╚██╔╝██║██╔══██╗██╔══██║
        ██║ ╚═╝ ██║██║  ██║██║ ╚═╝ ██║██████╔╝██║  ██║
        ╚═╝     ╚═╝╚═╝  ╚═╝╚═╝     ╚═╝╚═════╝ ╚═╝  ╚═╝

        mamba (0.22.1) supported by @QuantStack

        GitHub:  https://github.com/mamba-org/mamba
        Twitter: https://twitter.com/QuantStack

█████████████████████████████████████████████████████████████


Looking for: ['trimmomatic']

[+] 0.0s
conda-forge/linux-64 ━━━━━━━━━╸━━━━━━━━━━━━━   0.0 B /  ??.?MB @  ??.?MB/

### STEP 2: Setup Environment

Create a set of directories to store the reads, reference sequence files, and output files.


In [3]:
!cd $HOMEDIR
!echo $PWD
!mkdir -p data
!mkdir -p data/raw_fastq
!mkdir -p data/trimmed
!mkdir -p data/fastqc
!mkdir -p data/aligned
!mkdir -p data/reference

/home/jupyter/rnaseq-myco-notebook


### STEP 3: Downloading relevant FASTQ files using SRA Tools

Next we will need to download the relevant fastq files.

Because these files can be large, the process of downloading and extracting fastq files can be quite lengthy.

The sequence data for this tutorial comes from work by Cushman et al., <em><a href='https://www.ncbi.nlm.nih.gov/pmc/articles/PMC8191103/'>Increased whiB7 expression and antibiotic resistance in Mycobacterium chelonae carrying two prophages</a><em>.

We will be downloading the sample runs from this project using SRA tools, downloading from the NCBI's SRA (Sequence Run Archives).

However, first we need to find the associated accession numbers in order to download.


### STEP 3.1: Finding run accession numbers.

The SRA stores sequence data in terms of runs, (SRR stands for Sequence Read Run). To download runs, we will need the accession ID for each run we wish to download. 

The Cushman et al., project contains 12 runs. To make it easier, these are the run IDs associated with this project:

SRR13349122
SRR13349123
SRR13349124
SRR13349125
SRR13349126
SRR13349127
SRR13349128
SRR13349129
SRR13349130
SRR13349131
SRR13349132
SRR13349133


In this case, all these runs belong to the SRP (Sequence Run Project): SRP300216.

Sequence run experiments can be searched for using the SRA database on the NCBI website; and article specific sample run information can be found in the supplementary section of that article.

For instance, here, the the authors posted a link to the sequence data GSE (Gene Series number), <a href='https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE164210'>GSE164210</a>. This leads to the appropriate 'Gene Expression Omnibus' page where, among other useful files and information, the relevant SRA database link can be found. 

### STEP 3.2: Using the SRA-toolkit.

Now use the Sequence Run accession ID to download the sequence data.

In [4]:
!prefetch  -O data/raw_fastq/ SRR13349122


2022-06-01T17:31:23 prefetch.2.11.0: 1) Downloading 'SRR13349122'...
2022-06-01T17:31:23 prefetch.2.11.0:  Downloading via HTTPS...
2022-06-01T17:31:45 prefetch.2.11.0:  HTTPS download succeed
2022-06-01T17:31:47 prefetch.2.11.0:  'SRR13349122' is valid
2022-06-01T17:31:47 prefetch.2.11.0: 1) 'SRR13349122' was downloaded successfully
2022-06-01T17:31:47 prefetch.2.11.0: 'SRR13349122' has 0 unresolved dependencies


Notice the SRA archives sequence files in the SRA format. 

Typically genome workflows process data in the form of zipped or unzipped .fastq, or .fasta files

So before we move on, we need to convert the files from .sra to .fastq using the fastq-dump tool.

We will also compresss the fastq files to make them take less space, making them fastq.gz files.

In [5]:
#these are paired-end reads use the --split-files flag.
#to save space, compress to a zipped fastq file with the --gzip flag.
!fastq-dump -O data/raw_fastq/ --split-files --gzip data/raw_fastq/SRR13349122

Read 10827590 spots for data/raw_fastq/SRR13349122
Written 10827590 spots for data/raw_fastq/SRR13349122


### STEP 3.3 Downloading multiple files using the SRA-toolkit.

One may, as in our case, wish to download multiple runs at once.

To aid in this, SRA-tools supports batch downloading.

We can download multiple SRA files using a single line of code by creating a list of the SRA IDs we wish to download, and inputting that into the prefetch command.

For instance, one can first create a list of the 12 accession IDs for this project however they wish:

In [6]:
!for i in {22..33}; do echo "SRR133491$i"; done > data/raw_fastq/list_of_accesionIDS.txt
!cat data/raw_fastq/list_of_accesionIDS.txt

SRR13349122
SRR13349123
SRR13349124
SRR13349125
SRR13349126
SRR13349127
SRR13349128
SRR13349129
SRR13349130
SRR13349131
SRR13349132
SRR13349133


And then feed that list into the sra-toolkit prefetch command. Note, it may take some time to download all the fastq files.

In [7]:
!prefetch -O data/raw_fastq/ --option-file data/raw_fastq/list_of_accesionIDS.txt


2022-06-01T17:37:37 prefetch.2.11.0: 1) 'SRR13349122' is found locally
2022-06-01T17:37:37 prefetch.2.11.0: 'SRR13349122' has 0 unresolved dependencies

2022-06-01T17:37:37 prefetch.2.11.0: 2) Downloading 'SRR13349123'...
2022-06-01T17:37:37 prefetch.2.11.0:  Downloading via HTTPS...
2022-06-01T17:37:57 prefetch.2.11.0:  HTTPS download succeed
2022-06-01T17:37:58 prefetch.2.11.0:  'SRR13349123' is valid
2022-06-01T17:37:58 prefetch.2.11.0: 2) 'SRR13349123' was downloaded successfully
2022-06-01T17:37:58 prefetch.2.11.0: 'SRR13349123' has 0 unresolved dependencies

2022-06-01T17:37:59 prefetch.2.11.0: 3) Downloading 'SRR13349124'...
2022-06-01T17:37:59 prefetch.2.11.0:  Downloading via HTTPS...
2022-06-01T17:38:19 prefetch.2.11.0:  HTTPS download succeed
2022-06-01T17:38:21 prefetch.2.11.0:  'SRR13349124' is valid
2022-06-01T17:38:21 prefetch.2.11.0: 3) 'SRR13349124' was downloaded successfully
2022-06-01T17:38:21 prefetch.2.11.0: 'SRR13349124' has 0 unresolved dependencies

2022-06-01

### STEP 3.3 Converting Multiple SRA files to Fastq

We used fastq-dump before to convert SRA files to fastq. However, fastq-dump does not have native batch compatibility.

There are various ways to get around this, for instance, commonly by using bash pipes or for-loops.

One can also attempt to speed up the process of converting SRA to fastq files, by using fasterq-dump from the SRA-Tools library, or 'parallel-fastq-dump' from the separate parralel-fastq-dump library.

There is more than one right way to do anything, but parallel-fastq-dump has the benefit of automatic zipping, whereas fasterq-dump does not have --gzip flag.

Below is an example of one way to batch convert SRA to fastq files, piping in the same accession list used to download the SRAs with prefetech, to specify the files to convert from sra to fastq.gz

The below code may take approximately 30 minutes to run.

In [8]:
!cat data/raw_fastq/list_of_accesionIDS.txt |xargs -I {} parallel-fastq-dump -O data/raw_fastq/ --tmpdir . --threads 8 --gzip --split-files --sra-id {}

2022-06-01 17:41:52,858 - SRR ids: ['SRR13349122']
2022-06-01 17:41:52,858 - extra args: ['--gzip', '--split-files']
2022-06-01 17:41:52,859 - tempdir: ./pfd__vcitxrg
2022-06-01 17:41:52,859 - CMD: sra-stat --meta --quick SRR13349122
2022-06-01 17:41:55,368 - SRR13349122 spots: 10827590
2022-06-01 17:41:55,369 - blocks: [[1, 1353448], [1353449, 2706896], [2706897, 4060344], [4060345, 5413792], [5413793, 6767240], [6767241, 8120688], [8120689, 9474136], [9474137, 10827590]]
2022-06-01 17:41:55,369 - CMD: fastq-dump -N 1 -X 1353448 -O ./pfd__vcitxrg/0 --gzip --split-files SRR13349122
2022-06-01 17:41:55,370 - CMD: fastq-dump -N 1353449 -X 2706896 -O ./pfd__vcitxrg/1 --gzip --split-files SRR13349122
2022-06-01 17:41:55,371 - CMD: fastq-dump -N 2706897 -X 4060344 -O ./pfd__vcitxrg/2 --gzip --split-files SRR13349122
2022-06-01 17:41:55,373 - CMD: fastq-dump -N 4060345 -X 5413792 -O ./pfd__vcitxrg/3 --gzip --split-files SRR13349122
2022-06-01 17:41:55,374 - CMD: fastq-dump -N 5413793 -X 6767

### STEP 4: Copy reference transcriptome files that will be used by Salmon using E-Direct

Salmon is a tool that aligns RNA-Seq reads to a transcriptome.

So we will need a transcriptome reference file.

To get one, we can search through the NCBI assembly database, find an assembly, and download transcriptome reference files from that assembly using FTP links.

For instance, we will use the <a href='https://www.ncbi.nlm.nih.gov/assembly/GCF_001632805.1'>ASM163280v1</a> refseq assembly, found by searching through the NCBI assembly database. The FTP links can be accessed through the website in various ways, one way is to click the 'FTP directory for RefSeq assembly' link, found under 'Access the data', section.

Alternatively, if one were inclined, one could take the less common route and perform this through the NCBI command line tool suite called 'Entrez Direct' (EDirect).

This is an intricate and complicated set of tools, with many ways to do any one thing.

Below is an example of using an eDirect search query with a refseq identifier to obtain the relevant FTP directory, and then using that to download desired reference files.

In [9]:
#parse for the ftp link and download the genome reference fasta file

!esearch -db assembly -query GCF_001632805.1 | efetch -format docsum \
| xtract -pattern DocumentSummary -element FtpPath_RefSeq \
| awk -F"/" '{print "curl -o data/reference/"$NF"_genomic.fna.gz " $0"/"$NF"_genomic.fna.gz"}' \
| bash

#parse for the ftp link and download the gtf reference fasta file

!esearch -db assembly -query GCF_001632805.1 | efetch -format docsum \
| xtract -pattern DocumentSummary -element FtpPath_RefSeq \
| awk -F"/" '{print "curl -o data/reference/"$NF"_genomic.gff.gz " $0"/"$NF"_genomic.gff.gz"}' \
| bash

# parse for the ftp link and download the feature-table reference file 
# (for later use for merging readcounts with gene names in R code).

!esearch -db assembly -query GCF_001632805.1 | efetch -format docsum \
| xtract -pattern DocumentSummary -element FtpPath_RefSeq \
| awk -F"/" '{print "curl -o data/reference/"$NF"_feature_table.txt.gz " $0"/"$NF"_feature_table.txt.gz"}' \
| bash


#unzip the compresseed fasta files

!gzip -d data/reference/*.gz --force

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1436k  100 1436k    0     0   762k      0  0:00:01  0:00:01 --:--:--  762k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  329k  100  329k    0     0   224k      0  0:00:01  0:00:01 --:--:--  224k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  222k  100  222k    0     0   167k      0  0:00:01  0:00:01 --:--:--  168k


Next we can use a tool called gffread to create a transcriptome reference file using the gtf and genome files we downloaded.

In [10]:
!gffread -w data/reference/GCF_001632805.1_transcriptome_reference.fa -g data/reference/GCF_001632805.1_ASM163280v1_genomic.fna data/reference/GCF_001632805.1_ASM163280v1_genomic.gff

FASTA index file data/reference/GCF_001632805.1_ASM163280v1_genomic.fna.fai created.


It is also recommended to include the full genome at the end of the transcriptome reference file, for the purpose of performing a 'decoy-aware' mapping, more information about which can be found in the Salmon documentation.

To alert the tool to the presence of this, we will also create a 'decoy file', which salmon needs pointed towards the full genome sequence in our transcriptome reference file.

In [11]:
!cat data/reference/GCF_001632805.1_transcriptome_reference.fa <(echo) data/reference/GCF_001632805.1_ASM163280v1_genomic.fna > data/reference/GCF_001632805.1_transcriptome_reference_w_decoy.fa
!echo "NZ_CP007220.1" > data/reference/decoys.txt

### STEP 5: Copy data file for Trimmomatic

One of trimmomatics functions is to trim sequence machine specific adapter sequences. These are usually within the trimmomatic installation directory in a folder called adapters.

Directories of packages within conda installations can be confusing, so in the case of using conda with trimmomatic, it may be easier to simply download or create a file with the relevant adapter sequencecs and store it in an easy to find directory.

In [12]:
!curl https://storage.googleapis.com/me-inbre-rnaseq-pipelinev2/config/TruSeq3-PE.fa --output TruSeq3-PE.fa
!head TruSeq3-PE.fa 

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    95  100    95    0     0    868      0 --:--:-- --:--:-- --:--:--   871
>PrefixPE/1
TACACTCTTTCCCTACACGACGCTCTTCCGATCT
>PrefixPE/2
GTGACTGGAGTTCAGACGTGTGCTCTTCCGATCT



### STEP 6: Run Trimmomatic
Trimmomatic will trim off any adapter sequences or low quality sequence it detects in the FASTQ files.

Using piping and our original list, it is possible to queue up a batch run of trimmomatic for all our files.

The below code may take approximately 35 minutes to run.

In [13]:
!cat data/raw_fastq/list_of_accesionIDS.txt | xargs -I {} trimmomatic PE -threads 2 'data/raw_fastq/{}_1.fastq.gz' 'data/raw_fastq/{}_2.fastq.gz' 'data/trimmed/{}_1_trimmed.fastq.gz' 'data/trimmed/{}_1_trimmed_unpaired.fastq.gz' 'data/trimmed/{}_2_trimmed.fastq.gz' 'data/trimmed/{}_2_trimmed_unpaired.fastq.gz' ILLUMINACLIP:TruSeq3-PE.fa:2:30:10:2:keepBothReads LEADING:3 TRAILING:3 MINLEN:36

TrimmomaticPE: Started with arguments:
 -threads 2 data/raw_fastq/SRR13349122_1.fastq.gz data/raw_fastq/SRR13349122_2.fastq.gz data/trimmed/SRR13349122_1_trimmed.fastq.gz data/trimmed/SRR13349122_2_trimmed.fastq.gz data/trimmed/SRR13349122_1_trimmed_unpaired.fastq.gz data/trimmed/SRR13349122_2_trimmed_unpaired.fastq.gz ILLUMINACLIP:TruSeq3-PE.fa:2:30:10:2:keepBothReads LEADING:3 TRAILING:3 MINLEN:36
Using PrefixPair: 'TACACTCTTTCCCTACACGACGCTCTTCCGATCT' and 'GTGACTGGAGTTCAGACGTGTGCTCTTCCGATCT'
ILLUMINACLIP: Using 1 prefix pairs, 0 forward/reverse sequences, 0 forward only sequences, 0 reverse only sequences
Quality encoding detected as phred33
Input Read Pairs: 10827590 Both Surviving: 10810267 (99.84%) Forward Only Surviving: 17297 (0.16%) Reverse Only Surviving: 0 (0.00%) Dropped: 26 (0.00%)
TrimmomaticPE: Completed successfully
TrimmomaticPE: Started with arguments:
 -threads 2 data/raw_fastq/SRR13349123_1.fastq.gz data/raw_fastq/SRR13349123_2.fastq.gz data/trimmed/SRR13349123_1_tri

### STEP 7: Run FastQC
FastQC is an invaluable tool that allows you to evaluate whether there are problems with a set of reads. For example, it will provide a report of whether there is any bias in the sequence composition of the reads.

If you notice the results of the trimming, you may have noted the sequences in the reverse reads were few, and largely unpaired. This may be an artifact from how the original sequencing process. This is okay, we can proceed from here simply using the forward reads.

The below code may take around 10 minutes to run.

In [14]:
!cat data/raw_fastq/list_of_accesionIDS.txt | xargs -I {} fastqc "data/trimmed/{}_1_trimmed.fastq.gz" -o data/fastqc/

Started analysis of SRR13349122_1_trimmed.fastq.gz
Approx 5% complete for SRR13349122_1_trimmed.fastq.gz
Approx 10% complete for SRR13349122_1_trimmed.fastq.gz
Approx 15% complete for SRR13349122_1_trimmed.fastq.gz
Approx 20% complete for SRR13349122_1_trimmed.fastq.gz
Approx 25% complete for SRR13349122_1_trimmed.fastq.gz
Approx 30% complete for SRR13349122_1_trimmed.fastq.gz
Approx 35% complete for SRR13349122_1_trimmed.fastq.gz
Approx 40% complete for SRR13349122_1_trimmed.fastq.gz
Approx 45% complete for SRR13349122_1_trimmed.fastq.gz
Approx 50% complete for SRR13349122_1_trimmed.fastq.gz
Approx 55% complete for SRR13349122_1_trimmed.fastq.gz
Approx 60% complete for SRR13349122_1_trimmed.fastq.gz
Approx 65% complete for SRR13349122_1_trimmed.fastq.gz
Approx 70% complete for SRR13349122_1_trimmed.fastq.gz
Approx 75% complete for SRR13349122_1_trimmed.fastq.gz
Approx 80% complete for SRR13349122_1_trimmed.fastq.gz
Approx 85% complete for SRR13349122_1_trimmed.fastq.gz
Approx 90% comp

### STEP 8: Run MultiQC
MultiQC reads in the FastQC reports and generate a compiled report for all the analyzed FASTQ files.

In [15]:
!multiqc -f data/fastqc/


  /// ]8;id=991048;https://multiqc.info\MultiQC]8;;\ 🔍 | v1.12

|           multiqc | Search path : /home/jupyter/rnaseq-myco-notebook/data/fastqc
|         searching | ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 24/24  stqc.html.html.html
|            fastqc | Found 12 reports
|           multiqc | Compressing plot data
|           multiqc | Report      : multiqc_report.html
|           multiqc | Data        : multiqc_data
|           multiqc | MultiQC complete


### STEP 9: Index the Transcriptome so that Trimmed Reads Can Be Mapped Using Salmon

In [16]:
!salmon index -t data/reference/GCF_001632805.1_transcriptome_reference_w_decoy.fa -p 8 -i data/reference/transcriptome_index --decoys data/reference/decoys.txt -k 31 --keepDuplicates

Version Info: ### PLEASE UPGRADE SALMON ###
### A newer version of salmon with important bug fixes and improvements is available. ####
###
The newest version, available at https://github.com/COMBINE-lab/salmon/releases
contains new features, improvements, and bug fixes; please upgrade at your
earliest convenience.
###
Sign up for the salmon mailing list to hear about new versions, features and updates at:
https://oceangenomics.com/subscribe
index ["data/reference/transcriptome_index"] did not previously exist  . . . creating it
[2022-06-01 18:54:15.106] [jLog] [info] building index
out : data/reference/transcriptome_index
[2022-06-01 18:54:15.106] [puff::index::jointLog] [info] Running fixFasta

[Step 1 of 4] : counting k-mers

[2022-06-01 18:54:15.491] [puff::index::jointLog] [warning] There were 1 transcripts that would need to be removed to avoid duplicates.
[2022-06-01 18:54:15.491] [puff::index::jointLog] [info] Replaced 0 non-ATCG nucleotides
[2022-06-01 18:54:15.491] [puff::inde

### STEP 10: Run Salmon to Map Reads to Transcripts and Quantify Expression Levels
Salmon aligns the trimmed reads to the reference transcriptome and generates the read counts per transcript. In this analysis, each gene has a single transcript.

In [17]:
!cat data/raw_fastq/list_of_accesionIDS.txt | xargs -I {} salmon quant -i /home/jupyter/rnaseq-myco-notebook/data/reference/transcriptome_index -l SR -r "data/trimmed/{}_1_trimmed.fastq.gz" -p 8 --validateMappings -o "data/quants/{}_quant"

Version Info: ### PLEASE UPGRADE SALMON ###
### A newer version of salmon with important bug fixes and improvements is available. ####
###
The newest version, available at https://github.com/COMBINE-lab/salmon/releases
contains new features, improvements, and bug fixes; please upgrade at your
earliest convenience.
###
Sign up for the salmon mailing list to hear about new versions, features and updates at:
https://oceangenomics.com/subscribe
### salmon (selective-alignment-based) v1.7.0
### [ program ] => salmon 
### [ command ] => quant 
### [ index ] => { /home/jupyter/rnaseq-myco-notebook/data/reference/transcriptome_index }
### [ libType ] => { SR }
### [ unmatedReads ] => { data/trimmed/SRR13349122_1_trimmed.fastq.gz }
### [ threads ] => { 8 }
### [ validateMappings ] => { }
### [ output ] => { data/quants/SRR13349122_quant }
Logs will be written to data/quants/SRR13349122_quant/logs
[2022-06-01 18:54:46.249] [jointLog] [info] setting maxHashResizeThreads to 8
[2022-06-01 18:54:46.

### STEP 11: Report the top 10 most highly expressed genes in the samples

Top 10 most highly expressed genes in each wild-type sample.


In [18]:
!head data/quants/SRR13349122_quant/quant.sf -n 1
!sort -nrk 4,4 data/quants/SRR13349122_quant/quant.sf | head -10
!sort -nrk 4,4 data/quants/SRR13349123_quant/quant.sf | head -10
!sort -nrk 4,4 data/quants/SRR13349124_quant/quant.sf | head -10
!sort -nrk 4,4 data/quants/SRR13349125_quant/quant.sf | head -10
!sort -nrk 4,4 data/quants/SRR13349126_quant/quant.sf | head -10
!sort -nrk 4,4 data/quants/SRR13349127_quant/quant.sf | head -10

Name	Length	EffectiveLength	TPM	NumReads
rna-BB28_RS07080	117	3.947	366889.819745	16507.000
rna-BB28_RS07075	3116	2866.000	234649.688207	7665384.000
rna-BB28_RS07070	1516	1266.000	130544.614501	1883781.000
rna-BB28_RS17330	369	119.000	72555.739838	98414.000
gene-BB28_RS02220	204	9.377	8373.726433	895.000
gene-BB28_RS20695	231	14.032	6858.936071	1097.000
rna-BB28_RS09710	405	155.000	5466.039721	9657.000
gene-BB28_RS18745	300	51.326	3680.213081	2153.000
gene-BB28_RS18945	222	12.150	3530.894602	489.000
gene-BB28_RS24750	102	3.543	3392.339786	137.000
sort: write failed: 'standard output': Broken pipe
sort: write error
rna-BB28_RS07080	117	3.947	348464.132877	16529.000
rna-BB28_RS07075	3116	2866.000	224110.719521	7718494.000
rna-BB28_RS07070	1516	1266.000	132619.568495	2017600.000
rna-BB28_RS17330	369	119.000	76318.688035	109137.000
gene-BB28_RS02220	204	9.377	8723.549796	983.000
gene-BB28_RS20695	231	14.032	7312.353823	1233.000
rna-BB28_RS09710	405	155.000	6225.085179	11595.000
gene-BB28_R

Top 10 most highly expressed genes in the double lysogen samples.


In [19]:
!head data/quants/SRR13349122_quant/quant.sf -n 1
!sort -nrk 4,4 data/quants/SRR13349128_quant/quant.sf | head -10
!sort -nrk 4,4 data/quants/SRR13349129_quant/quant.sf | head -10
!sort -nrk 4,4 data/quants/SRR13349130_quant/quant.sf | head -10
!sort -nrk 4,4 data/quants/SRR13349131_quant/quant.sf | head -10
!sort -nrk 4,4 data/quants/SRR13349132_quant/quant.sf | head -10
!sort -nrk 4,4 data/quants/SRR13349133_quant/quant.sf | head -10

Name	Length	EffectiveLength	TPM	NumReads
rna-BB28_RS07080	117	3.947	453962.554944	24652.000
rna-BB28_RS07075	3116	2866.000	246655.204473	9725319.000
rna-BB28_RS07070	1516	1266.000	92462.501485	1610411.000
rna-BB28_RS17330	369	119.000	77568.389870	126990.000
gene-BB28_RS20695	231	14.032	4926.415147	951.000
rna-BB28_RS09710	405	155.000	4308.287043	9187.000
gene-BB28_RS02220	204	9.377	3790.573785	489.000
gene-BB28_RS14885	195	8.348	2603.559715	299.000
gene-BB28_RS06975	216	11.099	2541.095506	388.000
gene-BB28_RS21780	213	10.625	2223.501125	325.000
sort: write failed: 'standard output': Broken pipe
sort: write error
rna-BB28_RS07080	117	3.947	437397.002569	24834.000
rna-BB28_RS07075	3116	2866.000	237318.253525	9783257.000
rna-BB28_RS07070	1516	1266.000	95599.035600	1740858.000
rna-BB28_RS17330	369	119.000	83089.537568	142223.000
gene-BB28_RS20695	231	14.032	5400.553107	1090.000
rna-BB28_RS09710	405	155.000	5016.367275	11184.000
gene-BB28_RS02220	204	9.377	4122.229249	556.000
gene-BB28_RS14

### STEP 12: Report the expression of a putative acyl-ACP desaturase (BB28_RS16545) that was downregulated in the double lysogen relative to wild-type
A acyl-transferase was reported to be downregulated in the double lysogen as shown in the table of the top 20 upregulated and downregulated genes from the paper describing the study.
![RNA-Seq workflow](images/table-cushman.png)

Use `grep` to report the expression in the wild-type sample. The fields in the Salmon `quant.sf` file are as follows. The level of expression is reported in the Transcripts Per Million (`TPM`) and number of reads (`NumReads`) fields:  
`Name    Length  EffectiveLength TPM     NumReads`

In [20]:
!grep 'BB28_RS16545' data/quants/SRR13349122_quant/quant.sf
!grep 'BB28_RS16545' data/quants/SRR13349123_quant/quant.sf
!grep 'BB28_RS16545' data/quants/SRR13349124_quant/quant.sf
!grep 'BB28_RS16545' data/quants/SRR13349125_quant/quant.sf
!grep 'BB28_RS16545' data/quants/SRR13349126_quant/quant.sf
!grep 'BB28_RS16545' data/quants/SRR13349127_quant/quant.sf


gene-BB28_RS16545	987	737.000	53.330147	448.000
gene-BB28_RS16545	987	737.000	50.245685	445.000
gene-BB28_RS16545	987	737.000	42.489598	403.000
gene-BB28_RS16545	987	737.000	44.574169	441.000
gene-BB28_RS16545	987	737.000	52.578979	452.000
gene-BB28_RS16545	987	737.000	53.912280	487.000


Use `grep` to report the expression in the double lysogen sample. The fields in the Salmon `quant.sf` file are as follows. The level of expression is reported in the Transcripts Per Million (`TPM`) and number of reads (`NumReads`) fields:  
`Name    Length  EffectiveLength TPM     NumReads`

In [21]:
!grep 'BB28_RS16545' data/quants/SRR13349128_quant/quant.sf
!grep 'BB28_RS16545' data/quants/SRR13349129_quant/quant.sf
!grep 'BB28_RS16545' data/quants/SRR13349130_quant/quant.sf
!grep 'BB28_RS16545' data/quants/SRR13349131_quant/quant.sf
!grep 'BB28_RS16545' data/quants/SRR13349132_quant/quant.sf
!grep 'BB28_RS16545' data/quants/SRR13349133_quant/quant.sf

gene-BB28_RS16545	987	737.000	4.635462	47.000
gene-BB28_RS16545	987	737.000	6.508868	69.000
gene-BB28_RS16545	987	737.000	5.093176	37.000
gene-BB28_RS16545	987	737.000	5.906750	45.000
gene-BB28_RS16545	987	737.000	4.934811	43.000
gene-BB28_RS16545	987	737.000	6.218665	57.000


### STEP 12: Combine Genecounts to a Single Genecount File
Commonly, the readcounts for each sample are combined into a single table, where the rows contain the gene ID, and the columns identify the sample.

In [22]:
##first merge salmon files by number of reads.
!salmon quantmerge --column numreads --quants data/quants/*_quant -o data/quants/merged_quants.txt
##optinally we can rename the columns
!sed -i "1s/.*/Name\tSRR13349122\tSRR13349123\tSRR13349124\tSRR13349125\tSRR13349126\tSRR13349127\tSRR13349128\tSRR13349129\tSRR13349130\tSRR13349131\tSRR13349132\tSRR13349133/" data/quants/merged_quants.txt

##for further formatting, it may be easier in our r-code to later merge
##if we remove the gene- and rna- prefix
!sed -i "s/gene-//" data/quants/merged_quants.txt
!sed -i "s/rna-//" data/quants/merged_quants.txt

print("An example of a combined genecount outputfile.")
!head data/quants/merged_quants.txt

Version Info: ### PLEASE UPGRADE SALMON ###
### A newer version of salmon with important bug fixes and improvements is available. ####
###
The newest version, available at https://github.com/COMBINE-lab/salmon/releases
contains new features, improvements, and bug fixes; please upgrade at your
earliest convenience.
###
Sign up for the salmon mailing list to hear about new versions, features and updates at:
https://oceangenomics.com/subscribe
[2022-06-01 18:59:28.839] [mergeLog] [info] samples: [ data/quants/SRR13349122_quant, data/quants/SRR13349123_quant, data/quants/SRR13349124_quant, data/quants/SRR13349125_quant, data/quants/SRR13349126_quant, data/quants/SRR13349127_quant, data/quants/SRR13349128_quant, data/quants/SRR13349129_quant, data/quants/SRR13349130_quant, data/quants/SRR13349131_quant, data/quants/SRR13349132_quant, data/quants/SRR13349133_quant ]
[2022-06-01 18:59:28.839] [mergeLog] [info] sample names : [ SRR13349122_quant, SRR13349123_quant, SRR13349124_quant, SRR133491

## <a name="workflow">Additional Workflows</a>

Now that you have read counts per gene, feel free to explore the R workflow which creates plots and analyses using these readcount files, or try other alternate workflows for creating read count files, such as using snakemake.


[Workflow One:](Tutorial_1.ipynb) A short introduction to downloading and mapping sequences to a transcriptome using Trimmomatic and Salmon. Here is a link to the YouTube video demonstrating the tutorial: <https://www.youtube.com/watch?v=NG1U7D4l31o&t=26s>.

[Workflow One (Extended):](Tutorial_1B_Extended.ipynb) An extended version of workflow one. Once you have got your feet wet, you can retry workflow one with this extended version that covers the entire dataset, and includes elaboration such as using SRA tools for sequence downloading, and examples of running batches of fastq files through the pipeline. This workflow may take around an hour to run.

[Workflow One (Using Snakemake):](Tutorial_2_Snakemake.ipynb) Using snakemake to run workflow one.

[Workflow Two (DEG Analysis):](Tutorial_3_DEG_Analysis.ipynb) Using Deseq2 and R to conduct clustering and differential gene expression analysis.


![RNA-Seq workflow](images/RNA-Seq_Notebook_Homepage.png)